## Examining the Effects of Additive Noise on Exponential Smoothing Forecast Accuracy

***

In [1]:
# general modules
import pandas as pd
import numpy as np
import sktime

# import exponential smoothing forecasting model
from sktime.forecasting.exp_smoothing import ExponentialSmoothing

# functions for transformation+forecasting pipeline
from sktime.forecasting.compose import TransformedTargetForecaster

# time series transformations
from sktime.transformations.series.detrend import ConditionalDeseasonalizer

##### the `helper_functions.py` file contains many custom functions we wrote to aid in our analysis
##### `full_coding_analysis` combines all of the following - train-test split data,
##### data protection, train models, compare accuracies, return accuracy results
from helper_functions import *

# suppress warnings from exponential smoothing model not converging
import warnings
warnings.filterwarnings('ignore')

In [2]:
# import weekly finance time series
Y = np.genfromtxt("../../Data/Train/Clean/weekly_finance_clean.csv", delimiter = ',', skip_header = 1)
Y = pd.DataFrame(Y)

In [3]:
# detrender = Detrender()
# detrended_series = [detrender.fit_transform(series) for _ , series in Y.iterrows()]
# detrended_series = [i+np.abs(np.min(i))+1.0 for i in detrended_series]
# Y = pd.concat(detrended_series, axis=1).T

***

## SES

In [4]:
# define forecasting model
# perform additive deseasonalization conditional on autocorrelation test for seasonality

forecaster = TransformedTargetForecaster(
    [
        #("deseasonalize", ConditionalDeseasonalizer(model="additive", sp=52)),
        ("forecast", ExponentialSmoothing(use_boxcox=False)),
    ]
)

In [5]:
results_dict_ses = {}
fcasts_ses = {}
fcasts_protected_ses = {}
tests = {}
num_stdevs = [1, 2]
horizons = [1, 20]

In [6]:
for n in num_stdevs:
    for h in horizons:
        idx = "h="+str(h)+", "+str(n)+" stan. devs"
        results_dict_ses[idx], tests[idx], fcasts_ses[idx], fcasts_protected_ses[idx] = full_coding_analysis(time_series_data=Y, 
                                                                                                             forecasting_model=forecaster, 
                                                                                                             forecast_horizon=h,
                                                                                                             num_stdev=n)

***
***

In [7]:
adjusted_up = fcasts_ses['h=20, 1 stan. devs'] < fcasts_protected_ses['h=20, 1 stan. devs']
adjusted_up = pd.concat([row for i, row in adjusted_up.iterrows()])

adjusted_down = fcasts_ses['h=20, 1 stan. devs'] > fcasts_protected_ses['h=20, 1 stan. devs']
adjusted_down = pd.concat([row for i, row in adjusted_down.iterrows()])

In [8]:
absolute_error = np.absolute(tests['h=20, 1 stan. devs'] - fcasts_ses['h=20, 1 stan. devs'])
absolute_error = pd.concat([row for i, row in absolute_error.iterrows()])

In [9]:
protected_absolute_error = np.absolute(tests['h=20, 1 stan. devs'] - fcasts_protected_ses['h=20, 1 stan. devs'])
protected_absolute_error = pd.concat([row for i, row in protected_absolute_error.iterrows()])

In [10]:
np.mean(absolute_error[adjusted_up])

0.05884192028913125

In [11]:
np.mean(absolute_error[adjusted_down])

0.03505460217496818

In [12]:
np.mean(protected_absolute_error[adjusted_up])

0.24282142166013315

In [13]:
np.mean(protected_absolute_error[adjusted_down])

0.055579050489302124

***
***

In [14]:
results_dict_ses

{'h=1, 1 stan. devs': {'Mean Accuracies': array([2.18, 2.18]),
  'Protected Mean Accuracies:': array([12.65, 12.65]),
  '% Change Mean accuracy:': array([-480.98, -480.98]),
  '% Change Median accuracy:': array([-203.37, -203.37]),
  '% Forecasted Points adjusted downward:': 65.24,
  '% Forecasted Points adjusted upward:': 34.760000000000005,
  '% Series with improved accuracy:': array([18.9, 18.9]),
  '% Series with reduced accuracy:': array([81.1, 81.1]),
  'Original Mean Absolute Error Upward Adjusted:': 2.92,
  'Original Mean Absolute Error Downward Adjusted:': 1.78,
  'Protected Mean Absolute Error Upward Adjusted:': 28.299999999999997,
  'Protected Mean Absolute Error Downward Adjusted:': 4.31},
 'h=20, 1 stan. devs': {'Mean Accuracies': array([4.45, 5.43]),
  'Protected Mean Accuracies:': array([12.98, 13.7 ]),
  '% Change Mean accuracy:': array([-191.78, -152.28]),
  '% Change Median accuracy:': array([-41.38, -40.4 ]),
  '% Forecasted Points adjusted downward:': 60.37000000000

***
***

## DES

In [15]:
# define forecasting model
# perform additive deseasonalization conditional on autocorrelation test for seasonality

forecaster = TransformedTargetForecaster(
    [
        #("deseasonalize", ConditionalDeseasonalizer(model="additive", sp=52)),
        ("forecast", ExponentialSmoothing(trend="additive", use_boxcox=False)),
    ]
)

In [16]:
results_dict_des = {}
fcasts_des = {}
fcasts_protected_des = {}
tests = {}
num_stdevs = [1, 2]
horizons = [1, 20]

In [17]:
for n in num_stdevs:
    for h in horizons:
        idx = "h="+str(h)+", "+str(n)+" stan. devs"
        results_dict_des[idx], tests[idx], fcasts_des[idx], fcasts_protected_des[idx] = full_coding_analysis(time_series_data=Y, 
                                                                                                             forecasting_model=forecaster, 
                                                                                                             forecast_horizon=h,
                                                                                                             num_stdev=n)

In [18]:
results_dict_des

{'h=1, 1 stan. devs': {'Mean Accuracies': array([2.1, 2.1]),
  'Protected Mean Accuracies:': array([11.34, 11.34]),
  '% Change Mean accuracy:': array([-441.05, -441.05]),
  '% Change Median accuracy:': array([-158.21, -158.21]),
  '% Forecasted Points adjusted downward:': 50.61,
  '% Forecasted Points adjusted upward:': 49.39,
  '% Series with improved accuracy:': array([21.95, 21.95]),
  '% Series with reduced accuracy:': array([78.05, 78.05]),
  'Original Mean Absolute Error Upward Adjusted:': 2.34,
  'Original Mean Absolute Error Downward Adjusted:': 1.8599999999999999,
  'Protected Mean Absolute Error Upward Adjusted:': 18.63,
  'Protected Mean Absolute Error Downward Adjusted:': 4.2299999999999995},
 'h=20, 1 stan. devs': {'Mean Accuracies': array([4.28, 5.24]),
  'Protected Mean Accuracies:': array([12.12, 12.88]),
  '% Change Mean accuracy:': array([-182.93, -145.85]),
  '% Change Median accuracy:': array([-70.47, -58.62]),
  '% Forecasted Points adjusted downward:': 54.4499999

***
***

## TES

In [19]:
# define forecasting model
# perform additive deseasonalization conditional on autocorrelation test for seasonality

forecaster = TransformedTargetForecaster(
    [
        ("forecast", ExponentialSmoothing(trend="additive",
                                          seasonal="additive",
                                          sp=52,
                                          damped_trend=False, 
                                          use_boxcox=False)),
    ]
)

In [20]:
results_dict_tes = {}
fcasts_tes = {}
fcasts_protected_tes = {}
tests = {}
num_stdevs = [1, 2]
horizons = [1, 20]

In [21]:
for n in num_stdevs:
    for h in horizons:
        idx = "h="+str(h)+", "+str(n)+" stan. devs"
        results_dict_tes[idx], tests[idx], fcasts_tes[idx], fcasts_protected_tes[idx] = full_coding_analysis(time_series_data=Y, 
                                                                                                             forecasting_model=forecaster, 
                                                                                                             forecast_horizon=h,
                                                                                                             num_stdev=n)

In [22]:
results_dict_tes

{'h=1, 1 stan. devs': {'Mean Accuracies': array([2.44, 2.44]),
  'Protected Mean Accuracies:': array([14.49, 14.49]),
  '% Change Mean accuracy:': array([-493.41, -493.41]),
  '% Change Median accuracy:': array([-400.52, -400.52]),
  '% Forecasted Points adjusted downward:': 42.68,
  '% Forecasted Points adjusted upward:': 57.32000000000001,
  '% Series with improved accuracy:': array([20.73, 20.73]),
  '% Series with reduced accuracy:': array([79.27, 79.27]),
  'Original Mean Absolute Error Upward Adjusted:': 3.15,
  'Original Mean Absolute Error Downward Adjusted:': 1.49,
  'Protected Mean Absolute Error Upward Adjusted:': 18.77,
  'Protected Mean Absolute Error Downward Adjusted:': 8.74},
 'h=20, 1 stan. devs': {'Mean Accuracies': array([4.75, 5.74]),
  'Protected Mean Accuracies:': array([14.08, 15.95]),
  '% Change Mean accuracy:': array([-196.55, -177.75]),
  '% Change Median accuracy:': array([-106.06, -114.1 ]),
  '% Forecasted Points adjusted downward:': 48.809999999999995,
  